**Paso 0: Librerias**

In [9]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings("ignore")
 
import joblib

**Paso 1: Limpieza avanzada y estandarización de variables**

In [10]:
# 📌 LIMPIEZA Y RENOMBRADO DE VARIABLES CLAVE
df = pd.read_csv(r"C:\Users\javid\OneDrive\Escritorio\Javidev\ProjectML_Prediccion_Salarios_IT\data\survey_results_public.csv", low_memory=False)

# Renombramos las columnas más importantes
df = df.rename(columns={
    "Employment": "Tipo_empleo",
    "RemoteWork": "Trabajo_remoto",
    "DevType": "Rol",
    "EdLevel": "Nivel_educativo",
    "YearsCodePro": "Anios_experiencia",
    "Country": "Pais",
    "OrgSize": "Tamano_empresa",
    "ConvertedCompYearly": "Salario_anual"
})

# Filtramos columnas relevantes
columnas_utiles = [
    "Tipo_empleo", "Trabajo_remoto", "Rol", "Nivel_educativo",
    "Anios_experiencia", "Pais", "Tamano_empresa", "Salario_anual"
]
df = df[columnas_utiles]

# Limpiamos y transformamos la experiencia profesional
df["Anios_experiencia"] = df["Anios_experiencia"].replace({
    "Less than 1 year": "0",
    "More than 50 years": "51"
})
df["Anios_experiencia"] = pd.to_numeric(df["Anios_experiencia"], errors="coerce")

# Agrupamos países poco frecuentes como "Otro"
top_paises = df["Pais"].value_counts().head(15).index
df["Pais"] = df["Pais"].apply(lambda x: x if x in top_paises else "Otro")

# Convertimos tipo_empleo en categorías resumidas
def simplificar_tipo_empleo(valor):
    if pd.isna(valor): return None
    if "full-time" in valor: return "Jornada completa"
    if "part-time" in valor: return "Media jornada"
    if "freelancer" in valor or "contractor" in valor: return "Autonomo"
    if "student" in valor: return "Estudiante"
    if "not employed" in valor: return "Desempleado"
    if "retired" in valor: return "Jubilado"
    return "Otro"

df["Tipo_empleo"] = df["Tipo_empleo"].apply(simplificar_tipo_empleo)

# Simplificamos modalidad de trabajo
df["Trabajo_remoto"] = df["Trabajo_remoto"].replace({
    "Remote": "Remoto",
    "In-person": "Presencial",
    "Hybrid (some remote, some in-person)": "Hibrido"
})

def mapear_roles(rol_raw):
    mapa_roles = {
        "Developer, full-stack": "Developer_Full_Stack",
        "Developer, back-end": "Developer_Back_End",
        "Developer, front-end": "Developoer_Front-End",
        "Developer, desktop or enterprise applications": "Developer_Desktop_Enterprise",
        "Developer, mobile": "Developer_Mobile",
        "Developer, embedded applications or devices": "Developer_Embedded_Devices",
        "Other (please specify)": "Otro_rol",
        "Data engineer": "Data_Engineer",
        "Engineering manager": "Engineering_Manager",
        "DevOps specialist": "DevOps_Specialist",
        "Data scientist or machine learning specialist": "Data_Scientist_ML",
        "Research & Development role": "Investigación_Desarrollo",
        "Academic researcher": "Investigador_Academico",
        "Cloud infrastructure engineer": "Cloud_Infrastructure_Engineer",
        "Senior Executive (C-Suite, VP, etc.)": "Senior_Executive",
    }

    if pd.isna(rol_raw):
        return []
    
    roles = [r.strip() for r in rol_raw.split(";")]
    roles_mapeados = [mapa_roles.get(r, "Otro") for r in roles]
    return roles_mapeados

df["Rol"] = df["Rol"].apply(mapear_roles)

# Simplificamos nivel educativo
def simplificar_nivel(nivel):
    if pd.isna(nivel): return None
    if "Bachelor" in nivel: return "Grado universitario"
    if "Master" in nivel: return "Master"
    if "Professional" in nivel or "Ph.D" in nivel or "Doctoral" in nivel: return "Doctorado"
    if "Secondary" in nivel: return "Secundaria"
    if "Primary" in nivel: return "Primaria"
    if "Associate" in nivel: return "Grado medio"
    if "Some college" in nivel: return "Universidad sin titulo"
    return "Otro"

df["Nivel_educativo"] = df["Nivel_educativo"].apply(simplificar_nivel)

# Eliminamos filas nulas y salarios excesivos
# df = df.dropna()
df = df[df["Salario_anual"] <= 300000]


**Paso 2: Procesamiento de la columna 'Rol' y 'Pais' y 'Tamano_empresa'**

In [11]:
df["Anios_experiencia"] = df["Anios_experiencia"].replace({
    "Less than 1 year": "0", "More than 50 years": "51"
})
df["Anios_experiencia"] = pd.to_numeric(df["Anios_experiencia"], errors="coerce")
df["Tamano_empresa"] = df["Tamano_empresa"].replace("I don’t know", pd.NA)

In [12]:
df = df.dropna()
df = df[df["Salario_anual"] <= 300000]

**Paso 2: Procesamiento de la columna 'Rol' y 'Pais' y 'Tamano_empresa'**

In [13]:
# Rol
roles_dummies = df["Rol"].str.get_dummies(sep=";")
top_roles = roles_dummies.sum().sort_values(ascending=False).head(15).index
df = pd.concat([df.drop(columns="Rol"), roles_dummies[top_roles]], axis=1)

# País
top_paises = df["Pais"].value_counts().head(15).index
df["Pais"] = df["Pais"].apply(lambda x: x if x in top_paises else "Otro_pais")
df = pd.get_dummies(df, columns=["Pais"])
if "Pais_Otro" in df.columns:
    df = df.drop(columns=["Pais_Otro"])

# Resto
df = pd.get_dummies(df, columns=["Tipo_empleo", "Trabajo_remoto", "Nivel_educativo"], drop_first=False)

orden_empresa = [
    "Just me - I am a freelancer, sole proprietor, etc.",
    "2 to 9 employees",
    "10 to 19 employees",
    "20 to 99 employees",
    "100 to 499 employees",
    "500 to 999 employees",
    "1,000 to 4,999 employees",
    "5,000 to 9,999 employees",
    "10,000 or more employees"
]
df["Tamano_empresa"] = df["Tamano_empresa"].apply(lambda x: orden_empresa.index(x))

**Paso 3: Codificación de columnas categóricas**

In [14]:
for col in df.columns:
    if df[col].dtype == bool:
        df[col] = df[col].astype(int)

**Paso 4: Limpieza y preparación final**

In [15]:
# Preparar datos finales
X = df.drop(columns="Salario_anual")
X.columns = [re.sub(r"[^a-zA-Z0-9]", "_", col) for col in X.columns]
y = df["Salario_anual"]

In [16]:
# 📌 Sincronizar nombres de variables para compatibilidad con Streamlit

# Normalizamos textos antes de get_dummies
def limpiar_texto(x):
    if isinstance(x, str):
        x = x.strip()
        x = x.replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u").replace("ñ", "n")
        x = x.replace("_", " ")
        return x
    return x

df["Tipo_empleo"] = df["Tipo_empleo"].apply(limpiar_texto)
df["Trabajo_remoto"] = df["Trabajo_remoto"].apply(limpiar_texto)
df["Nivel_educativo"] = df["Nivel_educativo"].apply(limpiar_texto)
df["Pais"] = df["Pais"].apply(limpiar_texto)
df["Rol"] = df["Rol"].apply(lambda roles: [limpiar_texto(r) for r in roles] if isinstance(roles, list) else [])

# Aplicar get_dummies correctamente
X = pd.get_dummies(df.drop(columns="Salario_anual"), columns=["Tipo_empleo", "Trabajo_remoto", "Nivel_educativo", "Pais"], prefix_sep="", dtype=int)

# Explode roles y aplicar get_dummies a rol
roles_exploded = df.explode("Rol")
roles_dummies = pd.get_dummies(roles_exploded["Rol"], prefix="", prefix_sep="", dtype=int)
roles_sumados = roles_dummies.groupby(level=0).sum()
X = X.join(roles_sumados)

y = df["Salario_anual"]


KeyError: 'Tipo_empleo'

**Paso 5: División en train/test**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

**Paso 6: Definición del grid de hiperparámetros**

In [ ]:
param_grid = {
    "iterations": [800],
    "learning_rate": [0.03],
    "depth": [6],
    "l2_leaf_reg": [3],
    "bagging_temperature": [0.2]
}

**Paso 7: GridSearchCV y entrenamiento de CatBoost**

In [ ]:
model = CatBoostRegressor(verbose=0, random_state=42)

grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring="r2",
    cv=3,
    verbose=2,
    n_jobs=-1
)

grid.fit(X_train, y_train)

print("Mejor configuración encontrada:")
print(grid.best_params_)


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Mejor configuración encontrada:
{'bagging_temperature': 0.2, 'depth': 6, 'iterations': 800, 'l2_leaf_reg': 3, 'learning_rate': 0.03}


**Paso 8: Evaluación contra test**

In [ ]:
# Evaluación en test
mejor_modelo = grid.best_estimator_
y_pred = mejor_modelo.predict(X_test)
r2 = r2_score(y_test, y_pred)

print(f"R² en test con mejores hiperparámetros: {r2:.4f}")


R² en test con mejores hiperparámetros: 0.5946


**Paso 9: Guardado de modelo**

In [ ]:
columnas_modelo = X.columns.tolist()
joblib.dump((mejor_modelo, columnas_modelo), "model.pkl")

['model.pkl']

In [ ]:
list(X.head().columns)

['Anios_experiencia',
 'Tamano_empresa',
 '__Developer_Full_Stack__',
 '__Developer_Back_End__',
 '__Otro__',
 '__Developoer_Front_End__',
 '__Developer_Desktop_Enterprise__',
 '__Developer_Mobile__',
 '__Developer_Embedded_Devices__',
 '__Data_Engineer__',
 '__Engineering_Manager__',
 '__DevOps_Specialist__',
 '__Data_Scientist_ML__',
 '__Investigaci_n_Desarrollo__',
 '__Investigador_Academico__',
 '__Cloud_Infrastructure_Engineer__',
 '__Senior_Executive__',
 'Pais_Australia',
 'Pais_Brazil',
 'Pais_Canada',
 'Pais_France',
 'Pais_Germany',
 'Pais_India',
 'Pais_Italy',
 'Pais_Netherlands',
 'Pais_Otro',
 'Pais_Otro_pais',
 'Pais_Poland',
 'Pais_Spain',
 'Pais_Sweden',
 'Pais_Ukraine',
 'Pais_United_Kingdom_of_Great_Britain_and_Northern_Ireland',
 'Pais_United_States_of_America',
 'Tipo_empleo_Autonomo',
 'Tipo_empleo_Jornada_completa',
 'Tipo_empleo_Media_jornada',
 'Trabajo_remoto_Hibrido',
 'Trabajo_remoto_Presencial',
 'Trabajo_remoto_Remoto',
 'Nivel_educativo_Doctorado',
 'Nive